In [2]:
#Load libraries
import os
import torch
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import glob

import gc
from collections import defaultdict
from shutil import copy

In [3]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])


#train_transforms = torchvision.transforms.Compose([
    #torchvision.transforms.Resize((70, 70)),
    #torchvision.transforms.RandomCrop((64, 64)),
    #torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      #])

In [5]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

--2022-08-14 01:24:50--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2022-08-14 01:24:51--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  14.7MB/s    in 5m 38s  

2022-08-14 01:30:30 (14.1 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [ ]:
!tar -xvzf food-101.tar.gz

In [7]:
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [ ]:
prepare_data('food-101/meta/train.txt', 'food-101/images', 'train')

In [ ]:
prepare_data('food-101/meta/test.txt', 'food-101/images', 'test')

In [10]:
#Dataloader

#Path for training and testing directory
train_path= r'train'
test_path= r'test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [11]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [12]:
print(classes)
print(len(classes))

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [13]:
class AllConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        
        self.net = torch.nn.Sequential(
                      torch.nn.Conv2d(in_channels=3,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1,
                                      bias=False),
                      torch.nn.BatchNorm2d(16),
                      torch.nn.ReLU(inplace=True),
                      torch.nn.Conv2d(in_channels=16,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(2, 2),
                                      padding=1,
                                      bias=False),   
                      torch.nn.BatchNorm2d(16),
                      torch.nn.ReLU(inplace=True),
                      torch.nn.Conv2d(in_channels=16,
                                      out_channels=32,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1,
                                      bias=False),        
                      torch.nn.BatchNorm2d(32),
                      torch.nn.ReLU(inplace=True),                        
                      torch.nn.Conv2d(in_channels=32,
                                      out_channels=32,
                                      kernel_size=(3, 3),
                                      stride=(2, 2),
                                      padding=1,
                                      bias=False),      
                      torch.nn.BatchNorm2d(32),
                      torch.nn.ReLU(inplace=True),       
                      torch.nn.Conv2d(in_channels=32,
                                      out_channels=64,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1,
                                      bias=False),   
                      torch.nn.BatchNorm2d(64),
                      torch.nn.ReLU(inplace=True),
                      torch.nn.Conv2d(in_channels=64,
                                      out_channels=64,
                                      kernel_size=(3, 3),
                                      stride=(2, 2),
                                      padding=1,
                                      bias=False),     
                      torch.nn.BatchNorm2d(64),
                      torch.nn.ReLU(inplace=True),
                      torch.nn.Conv2d(in_channels=64,
                                      out_channels=num_classes,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1,
                                      bias=False),    
                      torch.nn.BatchNorm2d(101),
                      torch.nn.ReLU(inplace=True),
                      torch.nn.AdaptiveAvgPool2d(1),
                      torch.nn.Flatten()
    )


    def forward(self, x):
        
        x = self.net(x)
        #probas = torch.softmax(x, dim=1)
        return x

In [14]:
model=AllConvNet(num_classes=101).to(device)
print(model)

AllConvNet(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [15]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=10
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [17]:
print(train_count,test_count)


75750 25250


In [18]:
from tqdm import tqdm

In [19]:
#Model training and saving best model

best_accuracy=0.0

for epoch in tqdm(range(0,num_epochs)):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in tqdm(enumerate(train_loader)):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    val_loss = 0.0
    test_accuracy=0.0
    for i, (images,labels) in tqdm(enumerate(test_loader)):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        loss1=loss_function(outputs,labels)
        loss1.backward()
        optimizer.step()

        val_loss+= loss1.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    val_loss=val_loss/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Validation Loss: '+str(val_loss)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:07,  7.99s/it]
2it [00:08,  3.64s/it]
3it [00:09,  2.24s/it]
4it [00:09,  1.59s/it]
5it [00:10,  1.22s/it]
6it [00:10,  1.00it/s]
7it [00:11,  1.17it/s]
8it [00:11,  1.32it/s]
9it [00:12,  1.44it/s]
10it [00:13,  1.54it/s]
11it [00:13,  1.60it/s]
12it [00:14,  1.65it/s]
13it [00:14,  1.68it/s]
14it [00:15,  1.70it/s]
15it [00:15,  1.72it/s]
16it [00:16,  1.74it/s]
17it [00:17,  1.74it/s]
18it [00:17,  1.74it/s]
19it [00:18,  1.73it/s]
20it [00:18,  1.74it/s]
21it [00:19,  1.75it/s]
22it [00:19,  1.77it/s]
23it [00:20,  1.78it/s]
24it [00:20,  1.80it/s]
25it [00:21,  1.81it/s]
26it [00:22,  1.79it/s]
27it [00:22,  1.81it/s]
28it [00:23,  1.79it/s]
29it [00:23,  1.78it/s]
30it [00:24,  1.78it/s]
31it [00:24,  1.78it/s]
32it [00:25,  1.78it/s]
33it [00:26,  1.78it/s]
34it [00:26,  1.78it/s]
35it [00:27,  1.78it/s]
36it [00:27,  1.78it/s]
37it [00:28,  1.78it/s]
38it [00:28,  1.80it/s]
39it [00:29,  1.79it/s]
40it [00:29,  1

Epoch: 0 Train Loss: tensor(4.2326) Train Accuracy: 0.09032343234323433 Validation Loss: tensor(12159.4414) Test Accuracy: 0.009148514851485148



0it [00:00, ?it/s]
1it [00:00,  2.21it/s]
2it [00:00,  2.14it/s]
3it [00:01,  2.16it/s]
4it [00:01,  2.14it/s]
5it [00:02,  2.20it/s]
6it [00:02,  2.16it/s]
7it [00:03,  2.16it/s]
8it [00:03,  2.16it/s]
9it [00:04,  2.15it/s]
10it [00:04,  2.15it/s]
11it [00:05,  2.18it/s]
12it [00:05,  2.18it/s]
13it [00:05,  2.19it/s]
14it [00:06,  2.17it/s]
15it [00:06,  2.17it/s]
16it [00:07,  2.17it/s]
17it [00:07,  2.13it/s]
18it [00:08,  2.16it/s]
19it [00:08,  2.18it/s]
20it [00:09,  2.18it/s]
21it [00:09,  2.19it/s]
22it [00:10,  2.19it/s]
23it [00:10,  2.18it/s]
24it [00:11,  2.19it/s]
25it [00:11,  2.17it/s]
26it [00:11,  2.16it/s]
27it [00:12,  2.16it/s]
28it [00:12,  2.18it/s]
29it [00:13,  2.19it/s]
30it [00:13,  2.18it/s]
31it [00:14,  2.18it/s]
32it [00:14,  2.18it/s]
33it [00:15,  2.18it/s]
34it [00:15,  2.18it/s]
35it [00:16,  2.16it/s]
36it [00:16,  2.17it/s]
37it [00:17,  2.17it/s]
38it [00:17,  2.19it/s]
39it [00:17,  2.17it/s]
40it [00:18,  2.15it/s]
41it [00:18,  2.15it/s]
42it 

Epoch: 1 Train Loss: tensor(4.8252) Train Accuracy: 0.00998019801980198 Validation Loss: tensor(4.6331) Test Accuracy: 0.00998019801980198



0it [00:00, ?it/s]
1it [00:00,  2.30it/s]
2it [00:00,  2.19it/s]
3it [00:01,  2.18it/s]
4it [00:01,  2.21it/s]
5it [00:02,  2.21it/s]
6it [00:02,  2.23it/s]
7it [00:03,  2.24it/s]
8it [00:03,  2.25it/s]
9it [00:04,  2.24it/s]
10it [00:04,  2.23it/s]
11it [00:04,  2.25it/s]
12it [00:05,  2.26it/s]
13it [00:05,  2.25it/s]
14it [00:06,  2.25it/s]
15it [00:06,  2.23it/s]
16it [00:07,  2.22it/s]
17it [00:07,  2.23it/s]
18it [00:08,  2.22it/s]
19it [00:08,  2.23it/s]
20it [00:08,  2.21it/s]
21it [00:09,  2.23it/s]
22it [00:09,  2.23it/s]
23it [00:10,  2.23it/s]
24it [00:10,  2.24it/s]
25it [00:11,  2.23it/s]
26it [00:11,  2.24it/s]
27it [00:12,  2.21it/s]
28it [00:12,  2.24it/s]
29it [00:13,  2.23it/s]
30it [00:13,  2.25it/s]
31it [00:13,  2.23it/s]
32it [00:14,  2.21it/s]
33it [00:14,  2.23it/s]
34it [00:15,  2.24it/s]
35it [00:15,  2.24it/s]
36it [00:16,  2.25it/s]
37it [00:16,  2.27it/s]
38it [00:16,  2.27it/s]
39it [00:17,  2.26it/s]
40it [00:17,  2.26it/s]
41it [00:18,  2.25it/s]
42it 

...

Epoch: 7 Train Loss: tensor(4.6219) Train Accuracy: 0.009900990099009901 Validation Loss: tensor(4.6156) Test Accuracy: 0.009900990099009901



0it [00:00, ?it/s]
1it [00:00,  2.23it/s]
2it [00:00,  2.23it/s]
3it [00:01,  2.26it/s]
4it [00:01,  2.27it/s]
5it [00:02,  2.27it/s]
6it [00:02,  2.29it/s]
7it [00:03,  2.30it/s]
8it [00:03,  2.31it/s]
9it [00:03,  2.30it/s]
10it [00:04,  2.31it/s]
11it [00:04,  2.31it/s]
12it [00:05,  2.30it/s]
13it [00:05,  2.30it/s]
14it [00:06,  2.30it/s]
15it [00:06,  2.30it/s]
16it [00:06,  2.29it/s]
17it [00:07,  2.31it/s]
18it [00:07,  2.32it/s]
19it [00:08,  2.32it/s]
20it [00:08,  2.31it/s]
21it [00:09,  2.30it/s]
22it [00:09,  2.28it/s]
23it [00:10,  2.29it/s]
24it [00:10,  2.29it/s]
25it [00:10,  2.28it/s]
26it [00:11,  2.30it/s]
27it [00:11,  2.29it/s]
28it [00:12,  2.29it/s]
29it [00:12,  2.29it/s]
30it [00:13,  2.27it/s]
31it [00:13,  2.31it/s]
32it [00:13,  2.31it/s]
33it [00:14,  2.31it/s]
34it [00:14,  2.29it/s]
35it [00:15,  2.29it/s]
36it [00:15,  2.30it/s]
37it [00:16,  2.29it/s]
38it [00:16,  2.32it/s]
39it [00:16,  2.31it/s]
40it [00:17,  2.29it/s]
41it [00:17,  2.29it/s]
42it 

Epoch: 8 Train Loss: tensor(4.6228) Train Accuracy: 0.009900990099009901 Validation Loss: tensor(4.6154) Test Accuracy: 0.0100990099009901



0it [00:00, ?it/s]
1it [00:00,  2.20it/s]
2it [00:00,  2.26it/s]
3it [00:01,  2.27it/s]
4it [00:01,  2.28it/s]
5it [00:02,  2.30it/s]
6it [00:02,  2.30it/s]
7it [00:03,  2.30it/s]
8it [00:03,  2.28it/s]
9it [00:03,  2.28it/s]
10it [00:04,  2.30it/s]
11it [00:04,  2.29it/s]
12it [00:05,  2.29it/s]
13it [00:05,  2.31it/s]
14it [00:06,  2.33it/s]
15it [00:06,  2.30it/s]
16it [00:06,  2.31it/s]
17it [00:07,  2.29it/s]
18it [00:07,  2.29it/s]
19it [00:08,  2.28it/s]
20it [00:08,  2.27it/s]
21it [00:09,  2.27it/s]
22it [00:09,  2.27it/s]
23it [00:10,  2.29it/s]
24it [00:10,  2.29it/s]
25it [00:10,  2.29it/s]
26it [00:11,  2.31it/s]
27it [00:11,  2.33it/s]
28it [00:12,  2.33it/s]
29it [00:12,  2.31it/s]
30it [00:13,  2.33it/s]
31it [00:13,  2.30it/s]
32it [00:13,  2.30it/s]
33it [00:14,  2.32it/s]
34it [00:14,  2.31it/s]
35it [00:15,  2.20it/s]
36it [00:15,  1.96it/s]
37it [00:16,  1.81it/s]
38it [00:17,  1.73it/s]
39it [00:17,  1.86it/s]
40it [00:18,  1.98it/s]
41it [00:18,  2.07it/s]
42it 

Epoch: 9 Train Loss: tensor(4.6203) Train Accuracy: 0.010693069306930694 Validation Loss: tensor(4.6154) Test Accuracy: 0.009267326732673267
